# README    
1. generate a Keras Layer :AttLayer  
2. Use csv module to import data


# Result  
1)使用Dense层
acc在.73
roc在0.79
说明数据集没有问题

# Read Data

In [2]:
datafile  =  '/home/yinqijin/WorkSpace/2.RNA_Structure_Profile/Orig_data/pairs.csv'
import csv

In [3]:
with open(datafile) as csvfile:
    spamreader = csv.reader(csvfile)
    for  row in spamreader:
        csvkeys = row
        break
print csvkeys

['bin', 'enhancer_chrom', 'enhancer_distance_to_promoter', 'enhancer_end', 'enhancer_name', 'enhancer_start', 'label', 'promoter_chrom', 'promoter_end', 'promoter_name', 'promoter_start', 'window_end', 'window_start', 'window_chrom', 'window_name', 'interactions_in_window', 'active_promoters_in_window']


In [4]:
poscsv = {}
negcsv = {}
with open(datafile) as csvfile:
    reader = csv.DictReader(csvfile)
    for line in reader:
        if line['bin']  not in poscsv.keys():
            poscsv[line['bin']] =[]
        if line ['bin']  not in negcsv.keys():
            negcsv[line['bin']] =[]

        record=[ line['enhancer_chrom']+' '+
                line[ 'enhancer_start']+' '+
                line['enhancer_end']+' '+
                line['promoter_chrom']+' '+
                line['promoter_start']+' '+
                line['promoter_end']+' '+
                line['enhancer_name']+'-'+line['promoter_name']
                  ]
        if line['label']==str(1):
            poscsv[line['bin']].append(record)
        else:
            negcsv[line['bin']].append(record)

    for key in poscsv.keys():
        length = len(poscsv[key])
        index = range(len(negcsv[key]))
        np.random.shuffle(index)
        negcsv[key] = np.array(negcsv[key])
        negcsv[key] = negcsv[key][index[:length]]
        #print len(negcsv[key])

In [5]:
posdata =[]
negdata = []
for key in poscsv.keys():
    posdata = np.append(posdata,poscsv[key])
    negdata = np.append(negdata,negcsv[key])
assert  len(posdata) == len(negdata)
print  len(posdata),len(negdata)

2113 2113


In [6]:
from pyfasta import Fasta
genome = Fasta('/home/yinqijin/WorkSpace/DataHub/genome.fa')

In [7]:
#获得ｋｍｅｒ的函数
d2 = {'a':0, 'A':0, 'g':1, 'G':1, 'c':2, 'C':2, 't':3, 'T':3, 'N':0, 'n':0}
def seq_to_kspec(seq):
    mat = np.zeros((4096,1))
    k = 0
    if len(seq)<6:
        return mat
    for i in range(6):
        k = k*4 + d2[seq[i]]
    mat[k]+=1  
    for i in range(6,len(seq)):
        k = k - 4**5*d2[seq[i-6]]
        k = k*4 + d2[seq[i]]
        mat[k] += 1  
    return mat

In [8]:
PESeq=dict()
PESeq['index']=[]
PESeq['Pro-Seq']=[]
PESeq['Pro-Kmer']=[]
PESeq['Ehr-Seq']=[]
PESeq['Ehr-Kmer']=[]
PESeq['label'] =[]

In [9]:
#正样本
for index in range(len(posdata)):
    line = posdata[index]
    #print line
    line = line.split()
    ehr_seq = genome[line[0]][int(line[1]): int(line[2]) ]
    pro_seq = genome [line[3]][int(line[4]) :int(line[5])]
    index = line[6]
    
    if len(pro_seq)<6:
        print line
        continue
    if len(ehr_seq)<6:
        print line
        continue
        
    PESeq['index'].append(index )
    PESeq['Pro-Seq'].append(pro_seq)
    PESeq['Pro-Kmer'].append( seq_to_kspec(pro_seq))
    PESeq['Ehr-Seq'].append(ehr_seq)
    PESeq['Ehr-Kmer'].append(seq_to_kspec(ehr_seq))
    PESeq['label'].append(1)
print 'pos over'
print len(PESeq['label'])  

#负样本
  
for  index in range(len(negdata)):
    line = negdata[index]
    line = line.split()
    ehr_seq = genome[line[0]][int(line[1]): int(line[2]) ]
    pro_seq = genome [line[3]][int(line[4]) :int(line[5])]
    index = line[6]
    
    if len(pro_seq)<6:
        print line
        continue
    if len(ehr_seq)<6:
        print line
        continue
        
    PESeq['index'].append(index )
    PESeq['Pro-Seq'].append(pro_seq)
    PESeq['Pro-Kmer'].append( seq_to_kspec(pro_seq))
    PESeq['Ehr-Seq'].append(ehr_seq)
    PESeq['Ehr-Kmer'].append(seq_to_kspec(ehr_seq))
    PESeq['label'].append(0)
print 'neg over'

print len(PESeq['label'])

['chr17', '41080595', '41080600', 'chr17', '40728821', '40732200', 'GM12878|chr17:41080595-41080600-GM12878|chr17:40728821-40732200']
['chr3', '144015595', '144015600', 'chr3', '143689613', '143692895', 'GM12878|chr3:144015595-144015600-GM12878|chr3:143689613-143692895']
['chr6', '27857195', '27857200', 'chr6', '27803860', '27807953', 'GM12878|chr6:27857195-27857200-GM12878|chr6:27803860-27807953']
pos over
2110
['chr14', '23285600', '23285605', 'chr14', '24898401', '24901080', 'GM12878|chr14:23285600-23285605-GM12878|chr14:24898401-24901080']
['chr7', '25008400', '25008405', 'chr7', '24612524', '24614575', 'GM12878|chr7:25008400-25008405-GM12878|chr7:24612524-24614575']
['chr1', '45858795', '45858800', 'chr1', '46215504', '46216889', 'GM12878|chr1:45858795-45858800-GM12878|chr1:46215504-46216889']
['chr11', '62541195', '62541200', 'chr11', '62438320', '62440335', 'GM12878|chr11:62541195-62541200-GM12878|chr11:62438320-62440335']
['chr1', '160769395', '160769400', 'chr1', '160490425', 

In [10]:
print '-'*50
for item in PESeq.keys():
    print item, np.shape(PESeq[item])
print 'pos data' , sum(int( 1 ) if item ==1 else int(0) for item in PESeq['label'])
print 'neg data',sum(int( 1) if item ==0 else int(0) for item in PESeq['label'])



--------------------------------------------------
index (4217,)
Ehr-Kmer (4217, 4096, 1)
Ehr-Seq (4217,)
label (4217,)
Pro-Seq (4217,)
Pro-Kmer (4217, 4096, 1)
pos data 2110
neg data 2107


In [11]:
# Get X,Y with   limite  making positive and negative samples 's numbers is the same
X=[]
Y=[]
for index in range( len(PESeq['label'])):
    Y.append(PESeq['label'][index])
    X.append(np.append(PESeq['Pro-Kmer'][index],PESeq['Ehr-Kmer'][index],axis=1))
print np.shape(X)
print np.shape(Y)
X = np.array(X)
Y  = np.array(Y)


index= range(len(PESeq['label']))

(4217, 4096, 2)
(4217,)


In [12]:
from keras.utils.np_utils import to_categorical

In [13]:

VALIDATION_SPLIT =0.2

np.random.seed(1234)

np.random.shuffle(index)
nb_validation_samples = int(VALIDATION_SPLIT*len(Y))

In [14]:

x_train = X[index[: - nb_validation_samples]]
y_train = Y[index[:-nb_validation_samples]]
x_val = X[index[-nb_validation_samples:]]
y_val =Y[index[-nb_validation_samples:]]



In [15]:
y_train = to_categorical(y_train) #one-hot 编码labels
#y_val 其实没必要 one-hot编码
y_val = to_categorical(np.asarray(y_val))

In [16]:
print 'x_train_shape:',np.shape(x_train)
print 'y_train_shape:',np.shape(y_train)

x_train_shape: (3374, 4096, 2)
y_train_shape: (3374, 2)


In [ ]:
import hickle as hkl

In [ ]:
hkl.dump(x_val,'./Gen_data/13_x_val.hkl')
hkl.dump(y_val,'./Gen_data/13_y_val.hkl')
hkl.dump(x_train,'./Gen_data/13_x_train.hkl')
hkl.dump(y_train,'./Gen_data/13_y_train.hkl')

In [1]:
import hickle as hkl
x_val = hkl.load('./Gen_data/13_x_val.hkl')
y_val = hkl.load('./Gen_data/13_y_val.hkl')
x_train = hkl.load('./Gen_data/13_x_train.hkl')
y_train = hkl.load('./Gen_data/13_y_train.hkl')

# Make Network

In [2]:
from keras.layers import Embedding
from keras.layers import Dense,Input,Activation
from keras.layers import Embedding, LSTM, Bidirectional,GRU,InputLayer
from keras.models import Model,Sequential
from  keras.regularizers import ActivityRegularizer
from keras.layers.core import Dropout,Flatten

Using Theano backend.
Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5105)


In [3]:
from keras import layers

In [4]:
kmer_input = Input(shape=(4096,2), dtype='float32')
flat = Flatten()(kmer_input)

In [18]:
dense1 = Dense(256)(flat)

In [19]:
dense2 =Dense(32)(dense1)

In [20]:
preds = Dense(len( y_train[0]), activation='softmax',activity_regularizer= ActivityRegularizer(l2=0.005))(dense2)

In [21]:
model  = Model (kmer_input,preds)

In [22]:
print len(y_train[0])

2


In [23]:
model.compile(loss='mse',
              optimizer='rmsprop',
              metrics=['acc'])

print("model fitting - attention LSTM network")
model.summary()


model fitting - attention LSTM network
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 4096, 2)       0                                            
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 8192)          0           input_2[0][0]                    
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 256)           2097408     flatten_2[0][0]                  
____________________________________________________________________________________________________
dense_5 (Dense)                  (None, 32)            8224        dense_4[0][0]                    
____________________________________________________

In [24]:
model.fit(x_train, y_train, validation_data=(x_val, y_val), nb_epoch=50, batch_size=500)

Train on 3376 samples, validate on 843 samples
Epoch 1/50
3376/3376 [==============================] - 0s - loss: 0.9871 - acc: 0.5018 - val_loss: 0.9979 - val_acc: 0.4875
Epoch 2/50
3376/3376 [==============================] - 0s - loss: 0.9940 - acc: 0.5033 - val_loss: 0.9979 - val_acc: 0.4875
Epoch 3/50
3376/3376 [==============================] - 0s - loss: 0.9940 - acc: 0.5033 - val_loss: 0.9979 - val_acc: 0.4875
Epoch 4/50
3376/3376 [==============================] - 0s - loss: 0.9940 - acc: 0.5033 - val_loss: 0.9979 - val_acc: 0.4875
Epoch 5/50
3376/3376 [==============================] - 0s - loss: 0.9940 - acc: 0.5033 - val_loss: 0.9979 - val_acc: 0.4875
Epoch 6/50
3376/3376 [==============================] - 0s - loss: 0.9940 - acc: 0.5033 - val_loss: 0.9979 - val_acc: 0.4875
Epoch 7/50
3376/3376 [==============================] - 0s - loss: 0.9935 - acc: 0.5021 - val_loss: 0.9730 - val_acc: 0.5125
Epoch 8/50
3376/3376 [==============================] - 0s - loss: 1.0006 - ac

In [27]:
pro = model.predict_on_batch(x_val)

In [28]:
y_pred = [  1 if item1>item0 else 0 for item0,item1 in pro]

In [29]:
from sklearn import metrics

In [31]:
acc = metrics.accuracy_score(y_val[:,1],y_pred)
print acc

auc = metrics.roc_auc_score(y_val[:,1],pro[:,1])
print auc

f1_score = metrics.f1_score(y_val[:,1],y_pred)
print f1_score

0.710557532622
0.783443723529
0.66391184573


# Save&Search

In [ ]:
dir(model)

In [ ]:
model.__dict__

In [ ]:
model.save_weights('./Gen_data/10_Att-BLSTM.h5')

In [ ]:
model.load_weights('./Gen_data/10_Att-BLSTM.h5',by_name=True)